<a href="https://colab.research.google.com/github/farnaz-frd/search-engine-with-ranking/blob/main/Information_retrieval04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#this code is true and have a good snippet def and also have good GUI just I have to give it the Entire files
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

In [ ]:
# title = "stories"
alpha = 0.3

In [ ]:
#parsing
import os
from xml.etree import ElementTree
from bs4 import BeautifulSoup

dom = ElementTree.parse("D:/University/Term7CE/search Engine/HWs/Project/others project/Search-Engine-in-Python-master/WEBIRs/WebIR-157.xml")
urls = dom.findall('DOC')

def cleanMe(html):
    #title = html.title
    soup = BeautifulSoup(html, "html.parser") # create a new bs4 object from the html data loaded
    for script in soup(["script", "style"]): # remove all javascript and stylesheet code
        script.extract()
    # get text
    text = soup.get_text()
    if soup.title is not None:
        title = soup.title.string 
    else :
        title = "No title"
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    clean_text = ' '.join(BeautifulSoup(text, "html.parser").stripped_strings)
    clean_text = " ".join(clean_text.split())
#    title  = " ".join(title.split())
    return clean_text , title

def DataSeprate(Data) :
    
    for u in urls: 
        list=[]
        DOCID = u.find('DOCID').text
        URL   = u.find('URL').text
        html  = u.find('HTML').text
        [html , title] =cleanMe(html)
        list.append(DOCID )
        list.append(title )
        list.append(URL)
        list.append(html)
        
        Data.append(list)
    return Data


In [ ]:
Seperate_Data =[]
Seperate_Data = DataSeprate(Seperate_Data)

In [ ]:
dataset = []

for data in Seperate_Data :
    dataset.append((str(data[2]), str(data[1]),data[3]))

In [ ]:
dataset # our data must be : url , body , title

[('http://news.ut.ac.ir/nsite/fullstory/?id=10458',
  ' اخبار دانشگاه تهران:: عنوان خبر: از خدمات ارزنده دكتر رهبر تجلیل شد ',
  'اخبار دانشگاه تهران:: عنوان خبر: از خدمات ارزنده دكتر رهبر تجلیل شد اطلاعات تماس نسخه RSS کلیه مطالب معرفی جستجوی پیشرفته اخبار امروز : 0 / کل اخبار : 1705 / چهار شنبه 7 مرداد 1388 -- ساعت: 12:33 / بازدید کنندگان امروز : english فارسی news.ut.ac.ir صفحه اصلی روابط عمومي حوزه رياست حوزه معاونت ها معاونت آموزشي معاونت اداري و مالي معاونت بين المللي و دانشگاهي معاونت پژوهشي معاونت طرح و برنامه معاونت دانشجويي پرديس ها و دانشکده ها پرديس ابوريحان پرديس دانشکده هاي فني پرديس علوم پرديس قم پرديس كشاورزي و منابع طبيعي پرديس کيش پرديس هنرهاي زيبا دانشكده ادبيات و علوم انساني دانشكده اقتصاد دانشكده الهيات و معارف اسلامي دانشكده تربيت بدني و علوم ورزشي دانشکده جغرافيا دانشكده حقوق وعلوم سياسي دانشكده روان شناسي و علوم تربيتي دانشكده زبانها و ادبيات خارجي دانشكده علوم اجتماعي دانشکده کارآفريني دانشكده محيط زيست دانشكده مديريت دانشكده مطالعات جهان مراکز و مؤسسات تحقيقات

In [ ]:
N = len (dataset)
print(N)

238


In [ ]:
# def print_doc(id):
#     print(dataset[id])
#     file = open(dataset[id][0], 'r', encoding='cp1250')
#     text = file.read().strip()
#     file.close()
#     print(text)

In [ ]:
#Extracting Data
processed_text = []
processed_title = []

for data in Seperate_Data:
    text = data[3]
    processed_text.append(word_tokenize(str(text)))
    processed_title.append(word_tokenize(str(data[1])))

In [ ]:
print(processed_title)
print(len(processed_text))

[['اخبار', 'دانشگاه', 'تهران', ':', ':', 'عنوان', 'خبر', ':', 'از', 'خدمات', 'ارزنده', 'دكتر', 'رهبر', 'تجلیل', 'شد'], ['اخبار', 'دانشگاه', 'تهران', ':', ':', 'عنوان', 'خبر', ':', 'ايران', 'و', 'كميته', 'حقوق', 'كودك'], ['اخبار', 'دانشگاه', 'تهران', ':', ':', 'عنوان', 'خبر', ':', 'متدولوژي', 'GEM', 'در', 'دانشکده', 'کارآفريني', 'بررسي', 'مي', 'شود'], ['اخبار', 'دانشگاه', 'تهران', ':', ':', 'عنوان', 'خبر', ':', 'بازديد', 'عمومي', 'از', 'پرديس', 'کشاورزي', 'و', 'منابع', 'طبيعي', 'دانشگاه', 'تهران'], ['اخبار', 'دانشگاه', 'تهران', ':', ':', 'عنوان', 'خبر', ':'], ['اخبار', 'دانشگاه', 'تهران', ':', ':', 'عنوان', 'خبر', ':', 'کسب', 'رتبه', 'فصلنامه', 'مرکز', 'مطالعات', 'و', 'تحقيقات', 'زنان', 'دانشگاه', 'در', 'نشريات', 'ISC'], ['اخبار', 'دانشگاه', 'تهران', ':', ':', 'عنوان', 'خبر', ':', 'تسليت', 'ارتحال', 'آيت', 'الله', 'العظمي', 'بهجت'], ['اخبار', 'دانشگاه', 'تهران', ':', ':', 'عنوان', 'خبر', ':', 'فراخوان', 'انتخاب', 'كتاب', 'هاي', 'برگزيده', 'دانشگاهي'], ['اخبار', 'دانشگاه', 'تهران', ':', 

In [ ]:
#Calculating DF for all words
DF = {}

for i in range(len(Seperate_Data)):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [ ]:
total_vocab_size = len(DF)

In [ ]:
total_vocab_size

6220

In [ ]:
total_vocab = [x for x in DF]

In [ ]:
print(total_vocab[:100])

['اخبار', 'دانشگاه', 'تهران', ':', 'عنوان', 'خبر', 'از', 'خدمات', 'ارزنده', 'دكتر', 'رهبر', 'تجلیل', 'شد', 'اطلاعات', 'تماس', 'نسخه', 'RSS', 'کلیه', 'مطالب', 'معرفی', 'جستجوی', 'پیشرفته', 'امروز', '0', '/', 'کل', '1705', 'چهار', 'شنبه', '7', 'مرداد', '1388', '--', 'ساعت', '12:33', 'بازدید', 'کنندگان', 'english', 'فارسی', 'news.ut.ac.ir', 'صفحه', 'اصلی', 'روابط', 'عمومي', 'حوزه', 'رياست', 'معاونت', 'ها', 'آموزشي', 'اداري', 'و', 'مالي', 'بين', 'المللي', 'دانشگاهي', 'پژوهشي', 'طرح', 'برنامه', 'دانشجويي', 'پرديس', 'دانشکده', 'ابوريحان', 'هاي', 'فني', 'علوم', 'قم', 'كشاورزي', 'منابع', 'طبيعي', 'کيش', 'هنرهاي', 'زيبا', 'دانشكده', 'ادبيات', 'انساني', 'اقتصاد', 'الهيات', 'معارف', 'اسلامي', 'تربيت', 'بدني', 'ورزشي', 'جغرافيا', 'حقوق', 'وعلوم', 'سياسي', 'روان', 'شناسي', 'تربيتي', 'زبانها', 'خارجي', 'اجتماعي', 'کارآفريني', 'محيط', 'زيست', 'مديريت', 'مطالعات', 'جهان', 'مراکز', 'مؤسسات']


In [ ]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [ ]:
#Calculating TF-IDF for body, we will consider this as the actual tf-idf as we will add the title weight to this.
doc = 0

tf_idf = {}

for i in range(len(Seperate_Data)):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1


In [ ]:
#Calculating TF-IDF for Title
doc = 0

tf_idf_title = {}

for i in range(len(Seperate_Data)):
    
    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])

    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1)) #numerator is added 1 to avoid negative values
        
        tf_idf_title[doc, token] = tf*idf

    doc += 1

In [ ]:
tf_idf[(0,"تهران")]

0.0

In [ ]:
tf_idf_title[(0,"تهران")]

0.0

In [ ]:
#Merging the TF-IDF according to weights
for i in tf_idf:
    tf_idf[i] *= alpha

In [ ]:
for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]

In [ ]:
len(tf_idf)

66134

In [ ]:
#TF-IDF Matching Score Ranking
def matching_score(k, query):
    preprocessed_query = (query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    l = []
    
    for i in query_weights[:10]:
        l.append(i[0])
    
    print(l)

In [ ]:
# matching_score(10, "Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")
# matching_score(10, "دانشگاه تهران")

In [ ]:
#TF-IDF Cosine Similarity Ranking
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [ ]:
# N=len(Seperate_Data)
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [ ]:
print(len(Seperate_Data))
print(len(dataset))

238
238


In [ ]:
def gen_vector(tokens):
    N=len(dataset)
    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [ ]:
#make snippet
def makeSnippet(corpus , query2  , len_query):
    query2=query2.split()
    
    for i in range(len_query):
        query2=query2[:len_query-i]
        query2=' '.join(query2)
        sub_index = corpus.find(query2)
        if sub_index != -1 :
            if  sub_index > 72  :
                return  "..."+corpus[sub_index-71:sub_index-1] + corpus[sub_index:sub_index+70]+"..."
            else :
                return  corpus[sub_index:sub_index+140]+"..."
    return corpus[:140]

In [ ]:
from tkinter import *
from tkinter import messagebox
import sys
import os
import operator
import numpy as np
import tokeniser as tkn
import spellcheck as sc
import webbrowser
import time

In [ ]:
def cosine_similarity(k, query):
#     print("Cosine Similarity")
    preprocessed_query = (query)
    tokens = word_tokenize(str(preprocessed_query))
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
#     print(query_vector )
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    searchf=Tk()
    searchf.wm_title(query +'Beautiful Search')
    blank='           '
    blanklabel=Label( searchf,text=blank*20,font=("ComicSansMS", 10))
    label1 = Label( searchf,text=query+'\n\n',font=("ComicSansMS", 20))
    label1.pack()
    blanklabel.pack()
    def callbacks(event):
        webbrowser.open_new(event.widget.cget("text"))
    

    for i in out:
        snippet = makeSnippet(dataset[i][2] , query , len(query) )
        labl=Label(searchf,text=dataset[i][0]+"\n"+dataset[i][1]+"\n"+snippet+"\n"+"\n",font=("ComicSansMS", 12),justify=LEFT, fg="blue", cursor="hand2")
        labl.bind("<Button-1>", callbacks)
        labl.pack() 
        
#     for i in out:
#         print(dataset[i][0])
#         print(dataset[i][1])
#         snippet = makeSnippet(dataset[i][2] , query ,len(query))
#         print(snippet)
#         print("\n\n")


In [ ]:
top=Tk()
top.wm_title("Beautiful Search")
f = Frame(top, width=600,height=350)
f.pack(fill=X, expand=True)

e1=Entry(top,bd=6,width=40)
e1.insert(END, 'search here')
e1.place(relx=0.5, rely=0.35, anchor=CENTER)
b1=Button(top,text="Search",command= lambda: cosine_similarity(10, e1.get()))
b1.place(relx=0.5, rely=0.5, anchor=CENTER)
top.mainloop()

C:\Users\rasta\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#without GUI
cosine_similarity(k,query)

In [ ]:
# np.save('D2_saved', D)

In [ ]:
Q = cosine_similarity(20, "دانشگاه تهران رتبه دوم دانشگاه هاي خاورميانه را از آن خود کرد")

Cosine Similarity
http://news.ut.ac.ir/nsite/fullstory/?id=8484
 اخبار دانشگاه تهران:: عنوان خبر: دانشگاه تهران رتبه دوم دانشگاه هاي خاورميانه را از آن خود کرد 
دانشگاه تهران رتبه دوم دانشگاه هاي خاورميانه را از آن خود کرد اطلاعات تماس نسخه RSS کلیه مطالب معرفی جستجوی پیشرفته اخبار امروز : 0 / کل اخب...



http://news.ut.ac.ir/nsite/services/subservices/?serv=33&sgr=138
University Of Tehran News & Events :: اخبار دانشگاه تهران
...م پذيرفته شدگان دانشگاه تهران تاريخ ارسال :1/2/1386 | ساعت ارسال:13:29دانشگاه تهران رتبه دوم دانشگاه هاي خاورميانه را از آن خود کرد تاريخ ار...



http://news.ut.ac.ir/nsite/fullstory/?id=8218
 اخبار دانشگاه تهران:: عنوان خبر: رتبه وب سايت دانشگاه تهران در بين وب سايت دانشگاههاي خاورميانه و جهان 
دانشگاه تهران:: عنوان خبر: رتبه وب سايت دانشگاه تهران در بين وب سايت دانشگاههاي خاورميانه و جهان اطلاعات تماس نسخه RSS کلیه مطالب معرفی جستج...



http://news.ut.ac.ir/nsite/fullstory/?id=8637
 اخبار دانشگاه تهران:: عنوان خبر: سخنراني با عنوان "فدراسيون روسيه،آمريكا و

In [ ]:
# from tkinter import *
# from tkinter import ttk

# top=Tk()
# top.wm_title("Beautiful Search")
# f = Frame(top, width=600,height=350)
# f.pack(fill=X, expand=True)

# main_frame = Frame(top)
# main_frame.pack(fill = BOTH , expand=1)

# my_canvas=Canvas(main_frame)
# my_canvas.pack(side =LEFT , fill = BOTH , expand=1)

# my_scrollbar = ttk.Scrollbar(main_frame , orient = VERTICAL , command = my_canvas.yview )
# my_scrollbar.pack(side=RIGHT , fill = Y)

# my_canvas.configure(yscrollcommand = my_scrollbar.set)
# my_canvas.bind('<configure>' , lambda e:my_canvas.figure(scrollregion = my_canvas.bbox("all")))

# second_frame = Frame(my_canvas)

# my_canvas.create_window((0,0),window=second_frame , anchor="nw")

# for thing in range(100):
#     Button(top ,text=f'Button {thing}').grid(row=thing , column=0 , pady=10 , padx=10)

# top.mainloop()

In [ ]:
# from tkinter import *  
  
# top = Tk()  
# sb = Scrollbar(top)  
# sb.pack(side = RIGHT, fill = Y)  
  
# mylist = Listbox(top, yscrollcommand = sb.set )  
  
# for line in range(30):  
#     mylist.insert(END, "Number " + str(line))  
    
# mylist.pack( side = LEFT )  
# sb.config( command = mylist.yview )  
  
# mainloop()  